SIMPLE SENTIMENTAL ANALYSIS USING BAG OF WORDS
=====

## Introduction

This is a Sentiment prediction model which I built using Bag of Words algorithm, which aims to simply predict the sentiment of sentences.

## Dataset

The dataset contains 1,600,000 Twitter tweets . The tweets have been annotated (0 = negative, 4 = positive).

Link: https://www.kaggle.com/kazanova/sentiment140 

## Objective

Train a higher accurate model using bigger dataset. Before, I practiced this algorithm by using a 1000 lines dataset which created a very bad model. 

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
df=pd.read_csv(r'C:\Users\Asus\Downloads\training.1600000.processed.noemoticon.csv',encoding="latin-1")

In [54]:
df=df.drop(['NO_QUERY'],axis=1)

In [55]:
df.columns=['sentiment','id','sentday','user','sentence']

In [56]:
sen_list=[]
for i in df['sentiment']:
    if i==4: #Change the output value from 4 to 1 to closer the range.
        sen_list.append(1)
    if i==0:
        sen_list.append(0)
df['sentiment']=sen_list

In [6]:
df['sentiment'].unique()

array([0, 1], dtype=int64)

In [7]:
df.sample(10)

,sentiment,id,sentday,user,sentence
1587581,1,2190963914,Tue Jun 16 04:11:23 PDT 2009,corinaxjonas,im listening to LVaTT!! its hacking awesome!!
1136826,1,1976660678,Sat May 30 18:42:22 PDT 2009,Peulo,I think I got 9 hours of sleep.
1446571,1,2062525203,Sat Jun 06 23:29:15 PDT 2009,chris_cupcake,is in love with Christophe Dubois and his musi...
521909,0,2192562189,Tue Jun 16 07:13:14 PDT 2009,RBsmarchinglion,@EmClosk enjoy enjoy times a million. PS. I ha...
1289496,1,2002619781,Tue Jun 02 04:43:41 PDT 2009,urvi_rox,Bit soggy after run today. It was only 1/2 hou...
1268666,1,2000086289,Mon Jun 01 21:23:00 PDT 2009,jossjello,@AudreyLizeth Ahhh!!! Let's go
621781,0,2228942697,Thu Jun 18 15:04:56 PDT 2009,spencercofield,Seriously needs a massage
553778,0,2203703506,Wed Jun 17 00:20:17 PDT 2009,RK18,Phew !! Wait... where are my weekends?
1048639,1,1960192061,Fri May 29 07:34:16 PDT 2009,joshuar1313,@aliciadunaway me too baby. im starting my 2nd...
1172977,1,1980695580,Sun May 31 07:26:15 PDT 2009,sweetsmile36,"getting ready to sleep,after a busy night at w..."


In [8]:
#Create a new dataset with just only 2 columns 'sentiment' and 'sentence'
sentiment=df[['sentiment','sentence']]

In [9]:
sentiment.sample(10)

,sentiment,sentence
687559,0,http://thisiswhyyourefat.com/ - who the hell p...
1302603,1,Yes. I did!
703436,0,Omg the horse was not even close ~&lt;3~
1226224,1,@carole_hicks Well I'm coming from downtown Da...
470755,0,@super_seb29 Thanks. I don't have any camomile...
1068858,1,@amandabree I took the girls to a fabric store...
295272,0,i just burned the crap out of my hand...
306458,0,Woo! Party at my house! Too bad I'm not there....
1135744,1,@MiklBarton seriously! ask @sealabcore he got ...
270605,0,"well, a dozen grapes for midmorn snack just di..."


# PRE-PROCESSING

In [10]:
# stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Removing special characters and "trash"
import re
def preprocessor(text):
     # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

# Create some random texts for testing the function preprocessor()
print(preprocessor('aaaw,Aw'))

aaaw aw 


In [12]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer(text):
    return text.split() 

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

#Test
print(tokenizer('Ei yo solocococo momo'))
print(tokenizer_porter('Hi , loving hating bating'))

['Ei', 'yo', 'solocococo', 'momo']
['Hi', ',', 'love', 'hate', 'bate']


In [13]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split

X = df['sentence']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

# CREATE MODEL AND TRAIN

In [14]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenizer_porter, preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x000002F205EF17B8>,
                                 smooth_idf=True, stop_words=None,
                                 strip_accents=No...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x000002F20A8D12F0>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
        

# EVALUATE MODEL

In [15]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.79955
confusion matrix:
 [[94849 25050]
 [23058 97043]]
classification report:
               precision    recall  f1-score   support

           0       0.80      0.79      0.80    119899
           1       0.79      0.81      0.80    120101

    accuracy                           0.80    240000
   macro avg       0.80      0.80      0.80    240000
weighted avg       0.80      0.80      0.80    240000



In [23]:
twits = [
    "I used to not believe in religion, but now i am a happy Buddhist",
    "bad!#!#@@#!",
    "I 2edwdlove you"
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I used to not believe in religion, but now i am a happy Buddhist --> Negative, Positive = [0.34436768 0.65563232]
bad!#!#@@#! --> Negative, Positive = [0.9827388 0.0172612]
I 2edwdlove you --> Negative, Positive = [0.11782845 0.88217155]


# SAVE MODEL

In [24]:
# Using pickle to export the trained model
import pickle
import os

In [29]:
pickle.dump(clf, open(os.path.join(r'C:\Users\Asus\Downloads\Dataset', 'SentimentalPrediction.pkl'), 'wb'), protocol=4)

In [50]:
with open(r'C:\Users\Asus\Downloads\Dataset\SentimentalPrediction.pkl', 'rb') as function:
 function=pickle.load(function)

In [51]:
function

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x000002F205EF17B8>,
                                 smooth_idf=True, stop_words=None,
                                 strip_accents=No...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x000002F20A8D12F0>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
        